In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import FirefoxOptions
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import numpy as np
import re

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from Login import *

In [2]:
driver = get_driver('Firefox') # This might takes a while to boot up
time.sleep(5)

url = "https://reg.msu.edu/Courses/Search.aspx"
driver.get(url)
time.sleep(2)

element = driver.find_element(By.XPATH, "//option[@value='CMSE']")
element.click()
time.sleep(1)

element = driver.find_element(By.ID, "MainContent_btnSubmit")
element.click()
time.sleep(2)

In [3]:
results = driver.find_element(By.ID, "MainContent_divSearchResults").text
contents = results.split('\n')[1:]

reshaped_list = []
values = []
for i in range(len(contents)):
    if contents[i] == '' or "View all versions" in contents[i]:
        if len(values) != 0:
            reshaped_list.append(values)
        values = []
        continue
    if contents[i][-1] == ":":
        contents[i+1] = [contents[i][:-1], contents[i+1]]
        continue
    values.append(contents[i])

headers = set([element[:-1] for element in contents if element != '' and element[-1] == ":"])
headers.add('Course Name')
headers = list(headers)

In [4]:
complete_list = []
for values in reshaped_list:
    template = [np.nan] * len(headers)
    for i in range(len(values)):
        for j in range(len(headers)):
            if headers[j] == "Course Name":
                template[j] = values[0]
            if values[i][0] == headers[j]:
                template[j] = values[i][1]
    complete_list.append(template)

In [5]:
df = pd.DataFrame(complete_list, columns=headers)
df = df[['Course Name', 'Semester', 'Credits', 'Prerequisite', 'Recommended Background', 'Restrictions', 
         'Not open to students with credit in', 'Reenrollment Information', 'Description', 
         'Interdepartmental With', 'Administered By', 'Semester Alias', 'Effective Dates']]
df

,Course Name,Semester,Credits,Prerequisite,Recommended Background,Restrictions,Not open to students with credit in,Reenrollment Information,Description,Interdepartmental With,Administered By,Semester Alias,Effective Dates
0,CMSE 180 Introduction to Data Science,"Fall of every year, Spring of every year",Total Credits: 4 Lecture/Recitation/Discussi...,(MTH 124 or concurrently) or (MTH 132 or concu...,NaN,NaN,STT 301,NaN,Pervasiveness and utility of data in modern so...,Statistics and Probability,Statistics and Probability,NaN,FS19 - Open
1,CMSE 201 Computational Modeling and Data Anal...,"Fall of every year, Spring of every year",Total Credits: 4 Lecture/Recitation/Discussi...,MTH 124 or MTH 132 or MTH 152H or LB 118,NaN,NaN,NaN,NaN,Computational modeling using a wide variety of...,NaN,NaN,NSC 204,FS20 - Open
2,CMSE 202 Computational Modeling and Data Anal...,"Fall of every year, Spring of every year",Total Credits: 4 Lecture/Recitation/Discussi...,CMSE 201,NaN,NaN,NaN,NaN,Standard methods and tools for computational m...,NaN,NaN,NSC 205,FS20 - Open
3,CMSE 314 Matrix Algebra with Computational Ap...,"Fall of every year, Spring of every year, Summ...",Total Credits: 3 Lecture/Recitation/Discussi...,(MTH 133 or MTH 153H or LB 119) and (CMSE 201 ...,NaN,Not open to students in the Actuarial Science ...,NaN,NaN,Numerical methods in linear algebra with appli...,Mathematics,Mathematics,NaN,US19 - Open
4,CMSE 381 Fundamentals of Data Science Methods,"Fall of every year, Spring of every year",Total Credits: 4 Lecture/Recitation/Discussi...,(STT 180 and MTH 314 and CMSE 201 and STT 380)...,NaN,NaN,NaN,NaN,"Data science methods, including unsupervised l...",Statistics and Probability,"Computational Mathematics, Science, & Engineering",NaN,FS19 - Open
5,CMSE 382 Optimization Methods in Data Science,"Fall of every year, Spring of every year",Total Credits: 4 Lecture/Recitation/Discussi...,CMSE 202 and CMSE 381,NaN,NaN,NaN,NaN,"Concepts, mathematical foundations, methods, a...",NaN,NaN,NaN,FS19 - Open
6,CMSE 401 Methods for Parallel Computing,Spring of odd years,Total Credits: 4 Lecture/Recitation/Discussi...,(CMSE 202 and CSE 232) and (MTH 126 or MTH 133...,NaN,NaN,CSE 415,NaN,"Core principles, techniques, and use of parall...",NaN,NaN,NaN,FS21 - Open
7,CMSE 402 Data Visualization Principles and Te...,Spring of even years,Total Credits: 3 Lecture/Recitation/Discussi...,(CMSE 202) and (MTH 234 or MTH 254H or LB 220),NaN,NaN,NaN,NaN,"Core principles, methods, and techniques of ef...",NaN,NaN,NaN,FS19 - Open
8,CMSE 404 Introduction to Machine Learning,Fall of every year,Total Credits: 3 Lecture/Recitation/Discussi...,(CSE 331) and (STT 351 or STT 380 or STT 430 o...,Basic linear algebra,Open to juniors or seniors in the College of E...,NaN,NaN,Core principles and techniques for machine lea...,"Computer Science and Engineering, Statistics a...",Computer Science and Engineering,NaN,FS21 - Open
9,CMSE 410 Bioinformatics and Computational Bio...,Spring of every year,Total Credits: 3 Lecture/Recitation/Discussi...,{(CMSE 201 and LB 144 and LB 145) or (CMSE 201...,NaN,NaN,NaN,NaN,Computational approaches in modern biology wit...,"Biochemistry and Molecular Biology, Microbiolo...","Computational Mathematics, Science, & Engineering",NaN,FS20 - US25
